In [53]:
!pip3 install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
import torch
import torchvision
import matplotlib.pyplot as plt

from torch import nn
from torchvision import models, transforms
from torchvision.transforms import ToTensor
from torchinfo import summary

from PIL import Image
from collections import Counter
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

In [55]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Data

In [56]:
def custom_train_test_split(data, test_split=0.3):
    train_idx, test_idx = train_test_split(
        list(range(len(data))),
        test_size=test_split
    )

    dataset = {}
    dataset['train'] = torch.utils.data.Subset(data, train_idx)
    dataset['test'] = torch.utils.data.Subset(data, test_idx)

    return dataset

custom_transformer = transforms.Compose([
    # transforms.Resize((224, 224)),
    # transforms.RandomHorizontalFlip(),
    # transforms.ToTensor()
    models.ResNet152_Weights.IMAGENET1K_V2.transforms()
])

In [57]:
caltech_data = torchvision.datasets.Caltech256(
    root='.',
    transform=custom_transformer,
    download=True
)

caltech_data = torchvision.datasets.ImageFolder(
    root='./caltech256/256_ObjectCategories/',
    transform=custom_transformer
)

NUM_CLASSES = len(caltech_data.classes)

caltech_data = custom_train_test_split(caltech_data)

train_data = torch.utils.data.DataLoader(
    caltech_data['train'],
    batch_size=32,
    shuffle=True,
    drop_last=True,
    pin_memory=True
)
test_data = torch.utils.data.DataLoader(
    caltech_data['test'],
    batch_size=32
)

Files already downloaded and verified


In [58]:
train_features_batch, train_labels_batch = next(iter(train_data))
train_features_batch.shape, train_labels_batch.shape

(torch.Size([32, 3, 224, 224]), torch.Size([32]))

# Pretrained Model

In [59]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

weights = models.ResNet152_Weights.DEFAULT
model = models.resnet152(weights=weights)

for param in model.parameters():
  param.requires_grad = False

# Modify Last Layer
model.fc = nn.Sequential(
    nn.Linear(in_features=2048, out_features=1024),
    nn.ReLU(),
    nn.Linear(in_features=1024, out_features=1024),
    nn.ReLU(),
    nn.Linear(in_features=1024, out_features=1024),
    nn.ReLU(),
    nn.Linear(in_features=1024, out_features=NUM_CLASSES)
)

print(model.fc)
model = model.to(device)

summary(
    model=model,
    input_size=(32, 3, 224, 224), # example of [batch_size, color_channels, height, width],
    # input_data=torch.rand([32, 3, 224, 224], dtype=torch.float32),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"]
)

Sequential(
  (0): Linear(in_features=2048, out_features=1024, bias=True)
  (1): ReLU()
  (2): Linear(in_features=1024, out_features=1024, bias=True)
  (3): ReLU()
  (4): Linear(in_features=1024, out_features=1024, bias=True)
  (5): ReLU()
  (6): Linear(in_features=1024, out_features=257, bias=True)
)


Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [32, 3, 224, 224]    [32, 257]            --                   Partial
├─Conv2d (conv1)                         [32, 3, 224, 224]    [32, 64, 112, 112]   (9,408)              False
├─BatchNorm2d (bn1)                      [32, 64, 112, 112]   [32, 64, 112, 112]   (128)                False
├─ReLU (relu)                            [32, 64, 112, 112]   [32, 64, 112, 112]   --                   --
├─MaxPool2d (maxpool)                    [32, 64, 112, 112]   [32, 64, 56, 56]     --                   --
├─Sequential (layer1)                    [32, 64, 56, 56]     [32, 256, 56, 56]    --                   False
│    └─Bottleneck (0)                    [32, 64, 56, 56]     [32, 256, 56, 56]    --                   False
│    │    └─Conv2d (conv1)               [32, 64, 56, 56]     [32, 64, 56, 56]     (4,096)              False
│    │    

# Train and Test Function

In [60]:
image = torch.rand([32, 3, 224, 224])

torch.argmax(nn.functional.softmax(model(image.to(device)), dim=1), dim=1)

tensor([100, 178, 100, 178, 123, 178, 178, 178, 100, 178, 178, 178, 178, 100,
        100, 100, 178, 100, 178, 100, 178, 178, 100, 178, 178, 100, 178, 100,
        100, 178, 100, 100], device='cuda:0')

In [61]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct/len(y_pred)) * 100
    
    return acc

def train_function(model, train_data, loss_fn, optimizer):
    train_loss, train_accuracy = 0, 0

    model.train()

    for batch, (image, target) in enumerate(train_data):
        image, target = image.to(device), target.to(device)
        y_pred_logits = model(image)
        
        # train loss calculate
        tl = loss_fn(y_pred_logits, target)
        train_loss += tl.item()

        # train accuracy calculate
        y_pred = torch.argmax(nn.functional.softmax(y_pred_logits, dim=1), dim=1)
        tc = accuracy_fn(target, y_pred)
        train_accuracy += tc
        
        #optimizer zero grad --> backpropagation --> step
        optimizer.zero_grad()
        tl.backward()
        optimizer.step()

        if batch % 128 == 0:
            print(f"Looked at {batch * len(image)}/{len(train_data.dataset)} samples")

    
    train_loss /= len(train_data)
    train_accuracy /= len(train_data)

    return train_loss, train_accuracy


def test_function(model, test_data, loss_fn):
    test_loss, test_accuracy = 0, 0
    
    model.eval()
    with torch.inference_mode():
        for X_test, y_test in test_data:
            X_test, y_test = X_test.to(device), y_test.to(device)
            test_pred_logits = model(X_test)

            # test loss calculate
            test_loss += loss_fn(test_pred_logits, y_test).item()

            # test accuracy calculate
            y_pred = torch.argmax(nn.functional.softmax(test_pred_logits, dim=1), dim=1)
            test_accuracy += accuracy_fn(
                y_true = y_test,
                y_pred = y_pred
            )
            
        test_loss /= len(test_data)
        test_accuracy /= len(test_data)

    return test_loss, test_accuracy

In [62]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=0.001
)

In [63]:
torch.autograd.set_detect_anomaly(True)

In [64]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

epochs = 5
train_params = {
    "loss": [], 
    "accuracy": []
}
test_params = {
    "loss": [],
    "accuracy": []
}

for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}")

    train_loss, train_accuracy = train_function(model, train_data, loss_fn, optimizer)
    test_loss, test_accuracy = test_function(model, test_data, loss_fn)

    train_params["accuracy"].append(train_accuracy)
    train_params["loss"].append(train_loss)

    test_params["accuracy"].append(test_accuracy)
    test_params["loss"].append(test_loss)


print(train_params, test_params, sep='\n\n')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0
Looked at 0/21424 samples
Looked at 4096/21424 samples
Looked at 8192/21424 samples
Looked at 12288/21424 samples
Looked at 16384/21424 samples
Looked at 20480/21424 samples
Epoch: 1
Looked at 0/21424 samples
Looked at 4096/21424 samples
Looked at 8192/21424 samples
Looked at 12288/21424 samples
Looked at 16384/21424 samples
Looked at 20480/21424 samples
Epoch: 2
Looked at 0/21424 samples
Looked at 4096/21424 samples
Looked at 8192/21424 samples
Looked at 12288/21424 samples
Looked at 16384/21424 samples
Looked at 20480/21424 samples
Epoch: 3
Looked at 0/21424 samples
Looked at 4096/21424 samples
Looked at 8192/21424 samples
Looked at 12288/21424 samples
Looked at 16384/21424 samples
Looked at 20480/21424 samples
Epoch: 4
Looked at 0/21424 samples
Looked at 4096/21424 samples
Looked at 8192/21424 samples
Looked at 12288/21424 samples
Looked at 16384/21424 samples
Looked at 20480/21424 samples
{'loss': [2.2008739550373835, 0.7798571803778276, 0.5018986022748576, 0.3726733245362